<a href="https://colab.research.google.com/github/donbcolab/AIE3/blob/main/cnmc_leukemia_2019_brain_tumor_hf_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brain Tumor Image Dataset - Semantic Segmentation

- HF DataSet Loader



## Set up and Initial Checks

### Import Necessary Libraries and Define Constants

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -qU pyarrow==14.0.1 requests==2.31.0

In [ ]:
!pip install -qU datasets==2.11.0

In [ ]:
import os
import json
import pandas as pd
from datasets import Dataset, Features, ClassLabel, Value, Sequence, Image

In [ ]:
HF_DATASET_NAME = 'brain-tumor-image-dataset-semantic-segmentation'
SOURCE_JSON = "/content/drive/MyDrive/kaggle/datasets/brain-tumor-image-dataset-semantic-segmentation/train/_annotations.coco.json"
SOURCE_IMAGE_DIR = "/content/drive/MyDrive/kaggle/datasets/brain-tumor-image-dataset-semantic-segmentation/train"


### Initial Verification

In [ ]:
features = Features({
    'file_name': Value(dtype='string'),
    'image': Image(decode=True),
    'id': Value(dtype='int64'),
    'category_id': ClassLabel(names=['tumor', 'normal', 'other']),  # Add 'other' or adjust as needed
    'bbox': Sequence(feature=Value(dtype='float32'), length=4),
    'segmentation': Sequence(feature=Sequence(feature=Value(dtype='float32'), length=-1), length=-1),
    'area': Value(dtype='float32'),
    'iscrowd': Value(dtype='int64'),
    'height': Value(dtype='int64'),
    'width': Value(dtype='int64'),
    'date_captured': Value(dtype='string'),
    'license': Value(dtype='int64')
})

In [ ]:
def verify_source_data():
    if not os.path.exists(SOURCE_JSON):
        raise FileNotFoundError(f"JSON file not found: {SOURCE_JSON}")

    with open(SOURCE_JSON, 'r') as f:
        data = json.load(f)

    print(f"JSON file contains {len(data['images'])} images and {len(data['annotations'])} annotations")
    print("Sample of images:", data['images'][:2])
    print("Sample of annotations:", data['annotations'][:2])

In [ ]:
from tqdm.auto import tqdm

def load_data_to_df():
    with open(SOURCE_JSON, 'r') as f:
        data = json.load(f)

    images = pd.DataFrame(data['images'])
    annotations = pd.DataFrame(data['annotations'])

    df = pd.merge(images, annotations, left_on='id', right_on='image_id', suffixes=('', '_ann'))

    # Drop duplicate columns and rename as needed
    df = df.drop(columns=['id_ann', 'image_id'])

    # Add the full image path
    tqdm.pandas(desc="Processing image paths")
    df['image'] = df['file_name'].progress_apply(lambda x: os.path.join(SOURCE_IMAGE_DIR, x))

    # Ensure all required columns are present
    for column in features.keys():
        if column not in df.columns:
            df[column] = None

    return df

In [ ]:
import cv2
from tqdm.auto import tqdm

def create_hf_dataset(df, hf_dataset_name):
    # Convert 'image' column to image data
    def load_image(image_path):
        img = cv2.imread(image_path)
        if img is not None:
            return cv2.imencode('.jpg', img)[1].tobytes()
        return None

    tqdm.pandas(desc="Loading images")
    df['image'] = df['image'].progress_apply(load_image)

    dataset = Dataset.from_pandas(df, features=features)
    dataset.push_to_hub(hf_dataset_name)
    print(f"Dataset {hf_dataset_name} created and pushed to hub successfully.")

In [ ]:
from tqdm.auto import tqdm

# Execute the process
verify_source_data()
df = load_data_to_df()

# Add overall progress bar
with tqdm(total=3, desc="Creating dataset") as pbar:
    create_hf_dataset(df, HF_DATASET_NAME)
    pbar.update(1)

    # You can add more steps here if needed
    pbar.update(1)

    pbar.update(1)